# Generating Text using an LSTM Network (No libraries)

## Demo

We'll train an LSTM network built in pure numpy to generate Eminem lyrics. LSTMs are a fairly simple extension to neural networks, and they're behind a lot of the amazing achievements deep learning has made in the past few years.

## What is a Recurrent Network?

Recurrent nets are cool, they're useful for learning sequences of data. Input. Hidden state. Output. 
![alt text](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-rolled.png "Logo Title Text 1")

It has a weight matrix that connects input to hidden state. But also a weight matrix that connects hidden state to hidden state at previous time step.
![alt text](https://iamtrask.github.io/img/basic_recurrence_singleton.png "Logo Title Text 1")

So we could even think of it as the same feedforward network connecting to itself overtime (unrolled) since passing in
not just input in next training iteration but input + previous hidden state
![alt text](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png "Logo Title Text 1")

## The Problem with Recurrent Networks

If we want to predict the last word in the sentence "The grass is green", that's totally doable. 
![alt text](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-shorttermdepdencies.png "Logo Title Text 1")

But if we want to predict the last word in the sentence "I am French (2000 words later) i speak fluent French". We need to be able to remember long range dependencies. RNN's are bad at this. They forget the long term past easily.
![alt text](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-longtermdependencies.png "Logo Title Text 1")

This is called the "Vanishing Gradient Problem". The Gradient exponentially decays as its backpropagated
![alt text](http://slideplayer.com/slide/5251503/16/images/6/Recurrent+Neural+Networks.jpg "Logo Title Text 1")
![alt text](https://cdn-images-1.medium.com/max/1600/1*8JJ6sYleUtvUZR7TOyyFVg.png "Logo Title Text 1")

There are two factors that affect the magnitude of gradients - the weights and the activation functions (or more precisely, their derivatives) that the gradient passes through.If either of these factors is smaller than 1, then the gradients may vanish in time; if larger than 1, then exploding might happen. 

But there exists a solution! Enter the LSTM Cell.

## The LSTM Cell (Long-Short Term Memory Cell)

We've placed no constraints on how our model updates, so its knowledge can change pretty chaotically: at one frame it thinks the characters are in the US, at the next frame it sees the characters eating sushi and thinks they're in Japan, and at the next frame it sees polar bears and thinks they're on Hydra Island. 

This chaos means information quickly transforms and vanishes, and it's difficult for the model to keep a long-term memory. So what you’d like is for the network to learn how to update its beliefs (scenes without Bob shouldn't change Bob-related information, scenes with Alice should focus on gathering details about her), in a way that its knowledge of the world evolves more gently.

It replaces the normal RNN cell and uses an input, forget, and output gate. As well as a cell state
![alt text](https://www.researchgate.net/profile/Mohsen_Fayyaz/publication/306377072/figure/fig2/AS:398082849165314@1471921755580/Fig-2-An-example-of-a-basic-LSTM-cell-left-and-a-basic-RNN-cell-right-Figure.ppm "Logo Title Text 1")

![alt text](https://kijungyoon.github.io/assets/images/lstm.png "Logo Title Text 1")
These gates each have their own set of weight values. The whole thing is differentiable (meaning we compute gradients and update the weights using them) so we can backprop through it

We want our model to be able to know what to forget, what to remember. So when new a input comes in, the model first forgets any long-term information it decides it no longer needs. Then it learns which parts of the new input are worth using, and saves them into its long-term memory.

And instead of using the full long-term memory all the time, it learns which parts to focus on instead.

Basically, we need mechanisms for forgetting, remembering, and attention. That's what the LSTM cell provides us.

Whereas a vanilla RNN uses one equation to update its hidden state/memory:
![alt text](http://i.imgur.com/nT4VBPf.png "Logo Title Text 1")

Which piece of long term memory to remember and forget? 
We'll use new input and working memory to learn remember gate. 
Which part of new data should we use and save? 
Update working memory using attention vector. 

- The long-term memory, is usually called the cell state, 
- The working memory, is usually called the hidden state. This is analogous to the hidden state in vanilla RNNs.
- The remember vector, is usually called the forget gate (despite the fact that a 1 in the forget gate still means to keep the memory and a 0 still means to forget it),
- The save vector, is usually called the input gate (as it determines how much of the input to let into the cell state), 
- The focus vector, is usually called the output gate )

## Use cases


Video 
[![IMAGE ALT TEXT HERE](http://img.youtube.com/vi/mLxsbWAYIpw/0.jpg)](https://www.youtube.com/watch?time_continue=14&v=mLxsbWAYIpw)

The most popular application right now is actually in natural language processing
which involves sequential data such as words,  sentences, sound spectrogram, etc. So applications with translation,  sentiment analysis,  text generation, etc. 

In other less obvious areas there's also applications of lstm.  Such as for image classification (feeding each picture's pixel in row by row). And even for deepmind's deep Q Learning agents.

## Other great examples

Speech recognition Tensorflow - https://github.com/zzw922cn/Automatic_Speech_Recognition
LSTM visualization - https://github.com/HendrikStrobelt/LSTMVis

### Steps 

1. Build RNN class
2. Build LSTM Cell Class
3. Data Loading Functions
3. Training time!

![alt text](http://eric-yuan.me/wp-content/uploads/2015/06/5.jpg "Logo Title Text 1")

In [5]:
import numpy as np

class RecurrentNeuralNetwork:
    #input (word), expected output (next word), num of words (num of recurrences), array expected outputs, learning rate
    def __init__ (self, xs, ys, rl, eo, lr):
        #initial input (first word)
        self.x = np.zeros(xs)
        #input size 
        self.xs = xs
        #expected output (next word)
        self.y = np.zeros(ys)
        #output size
        self.ys = ys
        #weight matrix for interpreting results from LSTM cell (num words x num words matrix)
        self.w = np.random.random((ys, ys))
        #matrix used in RMSprop
        self.G = np.zeros_like(self.w)
        #length of the recurrent network - number of recurrences i.e num of words
        self.rl = rl
        #learning rate 
        self.lr = lr
        #array for storing inputs
        self.ia = np.zeros((rl+1,xs))
        #array for storing cell states
        self.ca = np.zeros((rl+1,ys))
        #array for storing outputs
        self.oa = np.zeros((rl+1,ys))
        #array for storing hidden states
        self.ha = np.zeros((rl+1,ys))
        #forget gate 
        self.af = np.zeros((rl+1,ys))
        #input gate
        self.ai = np.zeros((rl+1,ys))
        #cell state
        self.ac = np.zeros((rl+1,ys))
        #output gate
        self.ao = np.zeros((rl+1,ys))
        #array of expected output values
        self.eo = np.vstack((np.zeros(eo.shape[0]), eo.T))
        #declare LSTM cell (input, output, amount of recurrence, learning rate)
        self.LSTM = LSTM(xs, ys, rl, lr)
    
    #activation function. simple nonlinearity, convert nums into probabilities between 0 and 1
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    #the derivative of the sigmoid function. used to compute gradients for backpropagation
    def dsigmoid(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))    
    
    #lets apply a series of matrix operations to our input (curr word) to compute a predicted output (next word)
    def forwardProp(self):
        for i in range(1, self.rl+1):
            self.LSTM.x = np.hstack((self.ha[i-1], self.x))
            cs, hs, f, inp, c, o = self.LSTM.forwardProp()
            #store computed cell state
            self.ca[i] = cs
            self.ha[i] = hs
            self.af[i] = f
            self.ai[i] = inp
            self.ac[i] = c
            self.ao[i] = o
            self.oa[i] = self.sigmoid(np.dot(self.w, hs))
            self.x = self.eo[i-1]
        return self.oa
   
    
    def backProp(self):
        #update our weight matrices (Both in our Recurrent network, as well as the weight matrices inside LSTM cell)
        #init an empty error value 
        totalError = 0
        #initialize matrices for gradient updates
        #First, these are RNN level gradients
        #cell state
        dfcs = np.zeros(self.ys)
        #hidden state,
        dfhs = np.zeros(self.ys)
        #weight matrix
        tu = np.zeros((self.ys,self.ys))
        #Next, these are LSTM level gradients
        #forget gate
        tfu = np.zeros((self.ys, self.xs+self.ys))
        #input gate
        tiu = np.zeros((self.ys, self.xs+self.ys))
        #cell unit
        tcu = np.zeros((self.ys, self.xs+self.ys))
        #output gate
        tou = np.zeros((self.ys, self.xs+self.ys))
        #loop backwards through recurrences
        for i in range(self.rl, -1, -1):
            #error = calculatedOutput - expectedOutput
            error = self.oa[i] - self.eo[i]
            #calculate update for weight matrix
            #(error * derivative of the output) * hidden state
            tu += np.dot(np.atleast_2d(error * self.dsigmoid(self.oa[i])), np.atleast_2d(self.ha[i]).T)
            #Time to propagate error back to exit of LSTM cell
            #1. error * RNN weight matrix
            error = np.dot(error, self.w)
            #2. set input values of LSTM cell for recurrence i (horizontal stack of arrays, hidden + input)
            self.LSTM.x = np.hstack((self.ha[i-1], self.ia[i]))
            #3. set cell state of LSTM cell for recurrence i (pre-updates)
            self.LSTM.cs = self.ca[i]
            #Finally, call the LSTM cell's backprop, retreive gradient updates
            #gradient updates for forget, input, cell unit, and output gates + cell states & hiddens states
            fu, iu, cu, ou, dfcs, dfhs = self.LSTM.backProp(error, self.ca[i-1], self.af[i], self.ai[i], self.ac[i], self.ao[i], dfcs, dfhs)
            #calculate total error (not necesarry, used to measure training progress)
            totalError += np.sum(error)
            #accumulate all gradient updates
            #forget gate
            tfu += fu
            #input gate
            tiu += iu
            #cell state
            tcu += cu
            #output gate
            tou += ou
        #update LSTM matrices with average of accumulated gradient updates    
        self.LSTM.update(tfu/self.rl, tiu/self.rl, tcu/self.rl, tou/self.rl) 
        #update weight matrix with average of accumulated gradient updates  
        self.update(tu/self.rl)
        #return total error of this iteration
        return totalError
    
    def update(self, u):
        #vanilla implementation of RMSprop
        self.G = 0.9 * self.G + 0.1 * u**2  
        self.w -= self.lr/np.sqrt(self.G + 1e-8) * u
        return
    
    #this is where we generate some sample text after having fully trained our model
    #i.e error is below some threshold
    def sample(self):
         #loop through recurrences - start at 1 so the 0th entry of all arrays will be an array of 0's
        for i in range(1, self.rl+1):
            #set input for LSTM cell, combination of input (previous output) and previous hidden state
            self.LSTM.x = np.hstack((self.ha[i-1], self.x))
            #run forward prop on the LSTM cell, retrieve cell state and hidden state
            cs, hs, f, inp, c, o = self.LSTM.forwardProp()
            #store input as vector
            maxI = np.argmax(self.x)
            self.x = np.zeros_like(self.x)
            self.x[maxI] = 1
            self.ia[i] = self.x #Use np.argmax?
            #store cell states
            self.ca[i] = cs
            #store hidden state
            self.ha[i] = hs
            #forget gate
            self.af[i] = f
            #input gate
            self.ai[i] = inp
            #cell state
            self.ac[i] = c
            #output gate
            self.ao[i] = o
            #calculate output by multiplying hidden state with weight matrix
            self.oa[i] = self.sigmoid(np.dot(self.w, hs))
            #compute new input
            maxI = np.argmax(self.oa[i])
            newX = np.zeros_like(self.x)
            newX[maxI] = 1
            self.x = newX
        #return all outputs    
        return self.oa

![alt text](http://i.imgur.com/BUAVEZg.png "Logo Title Text 1")

In [6]:
class LSTM:
    # LSTM cell (input, output, amount of recurrence, learning rate)
    def __init__ (self, xs, ys, rl, lr):
        #input is word length x word length
        self.x = np.zeros(xs+ys)
        #input size is word length + word length
        self.xs = xs + ys
        #output 
        self.y = np.zeros(ys)
        #output size
        self.ys = ys
        #cell state intialized as size of prediction
        self.cs = np.zeros(ys)
        #how often to perform recurrence
        self.rl = rl
        #balance the rate of training (learning rate)
        self.lr = lr
        #init weight matrices for our gates
        #forget gate
        self.f = np.random.random((ys, xs+ys))
        #input gate
        self.i = np.random.random((ys, xs+ys))
        #cell state
        self.c = np.random.random((ys, xs+ys))
        #output gate
        self.o = np.random.random((ys, xs+ys))
        #forget gate gradient
        self.Gf = np.zeros_like(self.f)
        #input gate gradient
        self.Gi = np.zeros_like(self.i)
        #cell state gradient
        self.Gc = np.zeros_like(self.c)
        #output gate gradient
        self.Go = np.zeros_like(self.o)
    
    #activation function to activate our forward prop, just like in any type of neural network
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    #derivative of sigmoid to help computes gradients
    def dsigmoid(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))
    
    #tanh! another activation function, often used in LSTM cells
    #Having stronger gradients: since data is centered around 0, 
    #the derivatives are higher. To see this, calculate the derivative 
    #of the tanh function and notice that input values are in the range [0,1].
    def tangent(self, x):
        return np.tanh(x)
    
    #derivative for computing gradients
    def dtangent(self, x):
        return 1 - np.tanh(x)**2
    
    #lets compute a series of matrix multiplications to convert our input into our output
    def forwardProp(self):
        f = self.sigmoid(np.dot(self.f, self.x))
        self.cs *= f
        i = self.sigmoid(np.dot(self.i, self.x))
        c = self.tangent(np.dot(self.c, self.x))
        self.cs += i * c
        o = self.sigmoid(np.dot(self.o, self.x))
        self.y = o * self.tangent(self.cs)
        return self.cs, self.y, f, i, c, o
    
   
    def backProp(self, e, pcs, f, i, c, o, dfcs, dfhs):
        #error = error + hidden state derivative. clip the value between -6 and 6.
        e = np.clip(e + dfhs, -6, 6)
        #multiply error by activated cell state to compute output derivative
        do = self.tangent(self.cs) * e
        #output update = (output deriv * activated output) * input
        ou = np.dot(np.atleast_2d(do * self.dtangent(o)).T, np.atleast_2d(self.x))
        #derivative of cell state = error * output * deriv of cell state + deriv cell
        dcs = np.clip(e * o * self.dtangent(self.cs) + dfcs, -6, 6)
        #deriv of cell = deriv cell state * input
        dc = dcs * i
        #cell update = deriv cell * activated cell * input
        cu = np.dot(np.atleast_2d(dc * self.dtangent(c)).T, np.atleast_2d(self.x))
        #deriv of input = deriv cell state * cell
        di = dcs * c
        #input update = (deriv input * activated input) * input
        iu = np.dot(np.atleast_2d(di * self.dsigmoid(i)).T, np.atleast_2d(self.x))
        #deriv forget = deriv cell state * all cell states
        df = dcs * pcs
        #forget update = (deriv forget * deriv forget) * input
        fu = np.dot(np.atleast_2d(df * self.dsigmoid(f)).T, np.atleast_2d(self.x))
        #deriv cell state = deriv cell state * forget
        dpcs = dcs * f
        #deriv hidden state = (deriv cell * cell) * output + deriv output * output * output deriv input * input * output + deriv forget
        #* forget * output
        dphs = np.dot(dc, self.c)[:self.ys] + np.dot(do, self.o)[:self.ys] + np.dot(di, self.i)[:self.ys] + np.dot(df, self.f)[:self.ys] 
        #return update gradinets for forget, input, cell, output, cell state, hidden state
        return fu, iu, cu, ou, dpcs, dphs
            
    def update(self, fu, iu, cu, ou):
        #update forget, input, cell, and output gradients
        self.Gf = 0.9 * self.Gf + 0.1 * fu**2 
        self.Gi = 0.9 * self.Gi + 0.1 * iu**2   
        self.Gc = 0.9 * self.Gc + 0.1 * cu**2   
        self.Go = 0.9 * self.Go + 0.1 * ou**2   
        
        #update our gates using our gradients
        self.f -= self.lr/np.sqrt(self.Gf + 1e-8) * fu
        self.i -= self.lr/np.sqrt(self.Gi + 1e-8) * iu
        self.c -= self.lr/np.sqrt(self.Gc + 1e-8) * cu
        self.o -= self.lr/np.sqrt(self.Go + 1e-8) * ou
        return

In [7]:
def LoadText():
    #open text and return input and output data (series of words)
    with open("eminem.txt", "r") as text_file:
        data = text_file.read()
    text = list(data)
    outputSize = len(text)
    data = list(set(text))
    uniqueWords, dataSize = len(data), len(data) 
    returnData = np.zeros((uniqueWords, dataSize))
    for i in range(0, dataSize):
        returnData[i][i] = 1
    returnData = np.append(returnData, np.atleast_2d(data), axis=0)
    output = np.zeros((uniqueWords, outputSize))
    for i in range(0, outputSize):
        index = np.where(np.asarray(data) == text[i])
        output[:,i] = returnData[0:-1,index[0]].astype(float).ravel()  
    return returnData, uniqueWords, output, outputSize, data

#write the predicted output (series of words) to disk
def ExportText(output, data):
    finalOutput = np.zeros_like(output)
    prob = np.zeros_like(output[0])
    outputText = ""
    print(len(data))
    print(output.shape[0])
    for i in range(0, output.shape[0]):
        for j in range(0, output.shape[1]):
            prob[j] = output[i][j] / np.sum(output[i])
        outputText += np.random.choice(data, p=prob)    
    with open("output.txt", "w") as text_file:
        text_file.write(outputText)
    return

In [8]:
#Begin program    
print("Beginning")
iterations = 5000
learningRate = 0.001
#load input output data (words)
returnData, numCategories, expectedOutput, outputSize, data = LoadText()
print("Done Reading")
#init our RNN using our hyperparams and dataset
RNN = RecurrentNeuralNetwork(numCategories, numCategories, outputSize, expectedOutput, learningRate)

#training time!
for i in range(1, iterations):
    #compute predicted next word
    RNN.forwardProp()
    #update all our weights using our error
    error = RNN.backProp()
    #once our error/loss is small enough
    print("Error on iteration ", i, ": ", error)
    if error > -100 and error < 100 or i % 100 == 0:
        #we can finally define a seed word
        seed = np.zeros_like(RNN.x)
        maxI = np.argmax(np.random.random(RNN.x.shape))
        seed[maxI] = 1
        RNN.x = seed  
        #and predict some new text!
        output = RNN.sample()
        print(output)    
        #write it all to disk
        ExportText(output, data)
        print("Done Writing")
print("Complete")

Beginning
Done Reading
Error on iteration  1 :  5811032.205163492
Error on iteration  2 :  5776454.9846417
Error on iteration  3 :  5750098.033080126
Error on iteration  4 :  5728017.71163184
Error on iteration  5 :  5708418.602487503
Error on iteration  6 :  5690458.9569056565
Error on iteration  7 :  5673669.637964161
Error on iteration  8 :  5657758.799722952
Error on iteration  9 :  5642530.982208538
Error on iteration  10 :  5627848.149006289
Error on iteration  11 :  5613608.846924751
Error on iteration  12 :  5599736.157085972
Error on iteration  13 :  5586170.296813303
Error on iteration  14 :  5572863.853636436
Error on iteration  15 :  5559778.591908964
Error on iteration  16 :  5546883.241823327
Error on iteration  17 :  5534151.925462695
Error on iteration  18 :  5521563.0092675835
Error on iteration  19 :  5509098.249847982
Error on iteration  20 :  5496742.146473133
Error on iteration  21 :  5484481.442263127
Error on iteration  22 :  5472304.734382768
Error on iteration 

/Users/yoon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars
/Users/yoon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in less


Done Writing
Error on iteration  101 :  4558618.177925053
Error on iteration  102 :  4547126.984223817
Error on iteration  103 :  4535635.782464489
Error on iteration  104 :  4524144.593258579
Error on iteration  105 :  4512653.4153495
Error on iteration  106 :  4501162.247606281
Error on iteration  107 :  4489671.089011122
Error on iteration  108 :  4478179.938647692
Error on iteration  109 :  4466688.7956914455
Error on iteration  110 :  4455197.659400138
Error on iteration  111 :  4443706.529105431
Error on iteration  112 :  4432215.404206167
Error on iteration  113 :  4420724.284160962
Error on iteration  114 :  4409233.168482471
Error on iteration  115 :  4397742.056732175
Error on iteration  116 :  4386250.948514983
Error on iteration  117 :  4374759.843475381
Error on iteration  118 :  4363268.741293432
Error on iteration  119 :  4351777.641680676
Error on iteration  120 :  4340286.544377815
Error on iteration  121 :  4328795.4491511
Error on iteration  122 :  4317304.355789879


Error on iteration  272 :  2593621.425388092
Error on iteration  273 :  2582129.1983152917
Error on iteration  274 :  2570636.910068599
Error on iteration  275 :  2559144.557455391
Error on iteration  276 :  2547652.137120831
Error on iteration  277 :  2536159.645539716
Error on iteration  278 :  2524667.0790082226
Error on iteration  279 :  2513174.4336353736
Error on iteration  280 :  2501681.7053336985
Error on iteration  281 :  2490188.8898098636
Error on iteration  282 :  2478695.9825547994
Error on iteration  283 :  2467202.978833193
Error on iteration  284 :  2455709.8736727424
Error on iteration  285 :  2444216.661852803
Error on iteration  286 :  2432723.3378925617
Error on iteration  287 :  2421229.8960387004
Error on iteration  288 :  2409736.3302525845
Error on iteration  289 :  2398242.6341967075
Error on iteration  290 :  2386748.801220801
Error on iteration  291 :  2375254.8243472064
Error on iteration  292 :  2363760.6962555572
Error on iteration  293 :  2352266.4092670

Error on iteration  432 :  743125.3892097633
Error on iteration  433 :  731852.6390101704
Error on iteration  434 :  720618.8920076354
Error on iteration  435 :  709425.6366750783
Error on iteration  436 :  698274.0456522236
Error on iteration  437 :  687164.7443483912
Error on iteration  438 :  676099.5092671412
Error on iteration  439 :  665080.2794135521
Error on iteration  440 :  654108.334137939
Error on iteration  441 :  643184.2048132971
Error on iteration  442 :  632312.1194927075
Error on iteration  443 :  621490.6023954352
Error on iteration  444 :  610720.4652886214
Error on iteration  445 :  600004.719231369
Error on iteration  446 :  589345.537331633
Error on iteration  447 :  578746.5456138821
Error on iteration  448 :  568212.2007863261
Error on iteration  449 :  557743.9502860899
Error on iteration  450 :  547344.268914025
Error on iteration  451 :  537015.9508989871
Error on iteration  452 :  526756.9966568582
Error on iteration  453 :  516574.7248578587
Error on itera

...

Error on iteration  1094 :  6086.304823698259
Error on iteration  1095 :  5573.666826188879
Error on iteration  1096 :  6086.302884653207
Error on iteration  1097 :  5573.667492072511
Error on iteration  1098 :  6086.303397182896
Error on iteration  1099 :  5573.666005665411
Error on iteration  1100 :  6086.305523091921
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.06540345 0.00733928 0.08089019 ... 0.12002278 0.06724694 0.00822673]
 [0.00902869 0.00029031 0.01617088 ... 0.03051413 0.0168339  0.00069143]
 ...
 [0.00902869 0.00029031 0.01617088 ... 0.03051413 0.0168339  0.00069143]
 [0.00902869 0.00029031 0.01617088 ... 0.03051413 0.0168339  0.00069143]
 [0.00902869 0.00029031 0.01617088 ... 0.03051413 0.0168339  0.00069143]]
59
3359
Done Writing
Error on iteration  1101 :  5575.320996413106
Error on iteration  1102 :  6085.392508611996
Error on iteration  1103 :  5574.271293145052
Error on iteration  1104 :  6086.041660864411
Error on iteration  1105 :  5

Error on iteration  1252 :  6086.326179458226
Error on iteration  1253 :  5573.6897323621215
Error on iteration  1254 :  6086.245059142275
Error on iteration  1255 :  5573.742691915194
Error on iteration  1256 :  6086.22317095353
Error on iteration  1257 :  5573.729370662379
Error on iteration  1258 :  6086.264920949596
Error on iteration  1259 :  5573.672033353534
Error on iteration  1260 :  6086.32809679129
Error on iteration  1261 :  5573.620366732216
Error on iteration  1262 :  6086.363115099163
Error on iteration  1263 :  5573.610496775978
Error on iteration  1264 :  6086.35013761237
Error on iteration  1265 :  5573.6415343106555
Error on iteration  1266 :  6086.3077008357295
Error on iteration  1267 :  5573.683292398255
Error on iteration  1268 :  6086.272003755273
Error on iteration  1269 :  5573.703656640452
Error on iteration  1270 :  6086.267449239077
Error on iteration  1271 :  5573.692066822348
Error on iteration  1272 :  6086.291354431701
Error on iteration  1273 :  5573.6

Error on iteration  1410 :  6087.272452308518
Error on iteration  1411 :  5573.375750290106
Error on iteration  1412 :  6085.977516089003
Error on iteration  1413 :  5574.485945755889
Error on iteration  1414 :  6085.1695125157385
Error on iteration  1415 :  5574.793589424011
Error on iteration  1416 :  6085.3300666137975
Error on iteration  1417 :  5574.233786713579
Error on iteration  1418 :  6086.16020482635
Error on iteration  1419 :  5573.370728963194
Error on iteration  1420 :  6086.9381858762745
Error on iteration  1421 :  5572.883572019794
Error on iteration  1422 :  6087.111151886404
Error on iteration  1423 :  5573.054103929267
Error on iteration  1424 :  6086.6691475866855
Error on iteration  1425 :  5573.632287898541
Error on iteration  1426 :  6086.050630484934
Error on iteration  1427 :  5574.122284141045
Error on iteration  1428 :  6085.73455829416
Error on iteration  1429 :  5574.189614538256
Error on iteration  1430 :  6085.889147246593
Error on iteration  1431 :  5573

Error on iteration  1578 :  6086.340583103544
Error on iteration  1579 :  5573.650476299839
Error on iteration  1580 :  6086.300457311334
Error on iteration  1581 :  5573.686982375409
Error on iteration  1582 :  6086.271842675685
Error on iteration  1583 :  5573.70044468651
Error on iteration  1584 :  6086.273130476643
Error on iteration  1585 :  5573.68558548104
Error on iteration  1586 :  6086.297671152997
Error on iteration  1587 :  5573.658518796553
Error on iteration  1588 :  6086.323350670061
Error on iteration  1589 :  5573.6410667096725
Error on iteration  1590 :  6086.331460868499
Error on iteration  1591 :  5573.64395687455
Error on iteration  1592 :  6086.319523587043
Error on iteration  1593 :  5573.661214315936
Error on iteration  1594 :  6086.299972436243
Error on iteration  1595 :  5573.677662808414
Error on iteration  1596 :  6086.288340936131


KeyboardInterrupt: 